In [1]:
import sys
sys.path.append("../")
import recurrency_estimation as re
from tqdm.notebook import tqdm
import scipy.stats
from dask_jobqueue import SGECluster
from dask.distributed import Client, LocalCluster
import dask
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xarray as xr
import itertools
import time
import matplotlib
from scipy.optimize import curve_fit
from scipy.special import comb

from Session import SessionLite # required to unpickle data
from linear_model import PoolAcrossSessions, LinearModel, MultiSessionModel

/home/loidolt/RowlandEtAl/popping-off/popoff/popoff/loadpaths.py
/home/loidolt/RowlandEtAl/Vape


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
color_tt = {'hit': '#117733', 'miss': '#882255', 'fp': '#88CCEE', 'cr': '#DDCC77',
            'Hit': '#117733', 'Miss': '#882255', 'FP': '#88CCEE', 'CR': '#DDCC77',
            'urh': '#44AA99', 'arm': '#AA4499', 'spont': '#332288', 'prereward': '#332288', 
            'reward\nonly': '#332288', 'Reward\nonly': '#332288',
            'pre_reward': '#332288', 'Reward': '#332288', 'reward only': '#332288', 'rew. only': '#332288', 'hit&miss': 'k', 
            'fp&cr': 'k', 'photostim': sns.color_palette()[6], 'too_': 'grey',
            'hit_n1': '#b0eac9', 'hit_n2': '#5ab17f', 'hit_n3': '#117733',
            'miss_n1': '#a69098', 'miss_n2': '#985d76', 'miss_n3': '#882255',
            'hit_c1': '#b0eac9', 'hit_c2': '#5ab17f', 'hit_c3': '#117733',
            'miss_c1': '#a69098', 'miss_c2': '#985d76', 'miss_c3': '#882255'
            } 

In [4]:
import matplotlib
matplotlib.rcParams["xtick.labelsize"] = 10
matplotlib.rcParams["ytick.labelsize"] = 10
matplotlib.rcParams["axes.labelsize"] = 10
matplotlib.rcParams["axes.titlesize"]= 10
matplotlib.rcParams["legend.fontsize"] = 10
matplotlib.rcParams["legend.title_fontsize"] = 10

matplotlib.rcParams["axes.spines.right"] = True
matplotlib.rcParams["axes.spines.top"] = True
matplotlib.rcParams["axes.spines.left"] = True
matplotlib.rcParams["axes.spines.bottom"] = True

matplotlib.rcParams['axes.linewidth'] = 0.75
matplotlib.rcParams["axes.labelcolor"] = "black"
matplotlib.rcParams["axes.edgecolor"] = "black"
matplotlib.rcParams["xtick.color"] = "black"
matplotlib.rcParams["ytick.color"] = "black"

matplotlib.rcParams['xtick.major.size'] = 3
matplotlib.rcParams['xtick.major.width'] = 0.75
matplotlib.rcParams['xtick.minor.size'] = 3
matplotlib.rcParams['xtick.minor.width'] = 0.75

matplotlib.rcParams['ytick.major.size'] = 3
matplotlib.rcParams['ytick.major.width'] = 0.75
matplotlib.rcParams['ytick.minor.size'] = 3
matplotlib.rcParams['ytick.minor.width'] = 0.75

In [5]:
# load data for for latent factor and recurrence analysis
session_dict = re.load_data(remove_toosoon=True, label_urh_arm=True)

long post time
long post time
long post time
long post time
long post time
Mouse RL070, run 29  registered no-lick hit. changed to too soon
long post time
long post time
Mouse RL117, run 29  registered no-lick hit. changed to too soon
Mouse RL117, run 29  registered no-lick hit. changed to too soon
long post time
long post time
long post time
Mouse RL116, run 32  registered no-lick hit. changed to too soon
Mouse RL116, run 32  registered no-lick hit. changed to too soon
Mouse RL116, run 32  registered no-lick hit. changed to too soon
long post time
ALERT SESSIONS NOT SUBSAMPLED
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time
long post time


  9%|▉         | 1/11 [00:00<00:01,  7.74it/s]

long post time
{0: instance Mouse J064, run 10 of Session class, 1: instance Mouse J064, run 11 of Session class, 2: instance Mouse J064, run 14 of Session class, 3: instance Mouse RL070, run 28 of Session class, 4: instance Mouse RL070, run 29 of Session class, 5: instance Mouse RL117, run 26 of Session class, 6: instance Mouse RL117, run 29 of Session class, 7: instance Mouse RL117, run 30 of Session class, 8: instance Mouse RL123, run 22 of Session class, 9: instance Mouse RL116, run 32 of Session class, 10: instance Mouse RL116, run 33 of Session class}
URH and ARM trials have been labelled


100%|██████████| 11/11 [00:02<00:00,  4.73it/s]


In [ ]:
#per var explained -- LFA
n_sessions = len(session_dict.keys())
num_fits = 1000
num_factors = 5
num_frames = 65

hit_per_var_explained = np.zeros((n_sessions, num_fits, num_factors))
hit_per_var_explained[:] = np.nan

hit_cosine_similarity = np.zeros((n_sessions, num_fits, num_factors))
hit_cosine_similarity[:] = np.nan

hit_peristim_average = np.zeros((n_sessions, num_frames, num_factors))
hit_peristim_average[:] = np.nan

miss_per_var_explained = np.zeros((n_sessions, num_fits, num_factors))
miss_per_var_explained[:] = np.nan

miss_cosine_similarity = np.zeros((n_sessions, num_fits, num_factors))
miss_cosine_similarity[:] = np.nan

miss_peristim_average = np.zeros((n_sessions, num_frames, num_factors))
miss_peristim_average[:] = np.nan

i_session = -1

for session_name, session in session_dict.items():
    i_session += 1
    res = re.factor_analysis.per_var_explained_of_session(session, num_factors, num_fits, fa_type='lfa')
    
    hit_per_var_explained[i_session] = res[0]  
    miss_per_var_explained[i_session] = res[3]

np.save('../final_data/LFA.T_per_var_explained_hit_1000fits_5factors.npy', hit_per_var_explained)
np.save('../final_data/LFA.T_per_var_explained_miss_1000fits_5factors.npy', miss_per_var_explained)

In [57]:
## Connect to cluster for Recurrence analysis
try:
    client = await Client('tcp://172.9.50.181:46466', asynchronous=True)
except OSError:
    cluster = await LocalCluster()
    client = await Client(cluster, asynchronous=True)

In [62]:
## Recurrence analysis
num_fits = 1000 # Should be 1000, but 40 gives already good results 
n_factors_list = [21,22,23,24,25] # select factors to run here
num_bootstrap = 0
activity_type_list = ['shared', 'residual']
parameter_names = ('n_fact', 'session','activity_type')
parameter_values = (n_factors_list, list(session_dict.keys()), activity_type_list)

time_begin = time.time()

### Define LFA
session_dict_delayed={}
for session_name,session in session_dict.items():
    #### select S1 or S2 here
    session_dict_delayed[session_name] = dask.delayed(session.sel(area=1))

    
lfa_func = dask.delayed(re.factor_analysis.split_shared_residual_part, nout=2)
nested_session_dict = {}
for n_fact in n_factors_list:
    nested_session_dict[n_fact] = {key: {} for key in activity_type_list}
    for session_name,session in session_dict_delayed.items():
        if n_fact == 0:
            nested_session_dict[n_fact]['shared'][session_name] = session
            nested_session_dict[n_fact]['residual'][session_name] = session
            continue
        #### select LFA or PCA here
        session_shared, session_residual = lfa_func(session, n_fact=n_fact, fa_type='lfa')
        nested_session_dict[n_fact]['shared'][session_name] = session_shared
        nested_session_dict[n_fact]['residual'][session_name] = session_residual

### Define recurrency calculation        
delayed_dict = {}

recurrency_func = dask.delayed(re.variance_cc.var_cc_of_session)
for params_to_calc in itertools.product(*parameter_values):
    n_fact, session, activity_type = params_to_calc
    delayed_task = recurrency_func(nested_session_dict[n_fact][activity_type][session], num_bootstrap, num_fits=num_fits)
    delayed_dict[params_to_calc] = delayed_task

    
### Begin calculatation
future_dict = client.compute(delayed_dict)

In [ ]:
### Save results in xarray and to disc

result_dict = await future_dict.result()

res_dataset = xr.DataArray(data=None,  coords = [*zip(parameter_names, parameter_values)]+
                                                 [('variable',['var_cc', 'mean_var','mean_rho']),
                                                 ('trial_type',['hit', 'miss']),
                                                 ('sample',np.arange(num_bootstrap+1))])
for params_res, res in result_dict.items():
    n_fact, session, activity_type = params_res
    var_cc_hit, var_cc_miss, mean_var_hit, mean_var_miss, mean_rho_hit, mean_rho_miss = res
    res_dataset.loc[dict([*zip(parameter_names, params_res)])] = np.array([[var_cc_hit, var_cc_miss], [mean_var_hit, mean_var_miss], [mean_rho_hit, mean_rho_miss]])

res_dataset.to_netcdf(f"../final_data/ML_nfac21-25_results_{num_fits}fits.nc")

In [ ]:
# load data for tau_post and recurrence analysis
session_dict = re.load_data_prepost(remove_toosoon=True, label_urh_arm=True)

In [1]:
## HitAndMiss
num_fits = 1000 # Should be 1000, but 40 gives already good results 
n_factors_list = [0,5]
num_bootstrap = 0
activity_type_list = ['shared', 'residual']
parameter_names = ('n_fact', 'session', 'activity_type')
parameter_values = (n_factors_list, list(session_dict.keys()), activity_type_list)

time_begin = time.time()

### Define LFA
session_dict_delayed={}
for session_name,session in session_dict.items():
    session_dict_delayed[session_name] = dask.delayed(session.sel(area=1))

    
lfa_func = dask.delayed(re.factor_analysis.split_shared_residual_part, nout=2)
nested_session_dict = {}
for n_fact in n_factors_list:
    nested_session_dict[n_fact] = {key: {} for key in activity_type_list}
    for session_name,session in session_dict_delayed.items():
        if n_fact == 0:
            nested_session_dict[n_fact]['shared'][session_name] = session
            nested_session_dict[n_fact]['residual'][session_name] = session
            continue
        session_shared, session_residual = lfa_func(session, n_fact=n_fact, fa_type='lfa')
        nested_session_dict[n_fact]['shared'][session_name] = session_shared
        nested_session_dict[n_fact]['residual'][session_name] = session_residual

### Define recurrency calculation        
delayed_dict = {}
recurrency_func = dask.delayed(re.prepost_var_cc_and_taupost_of_session_TestOnly_HitAndMiss)
for params_to_calc in itertools.product(*parameter_values):
    n_fact, session, activity_type = params_to_calc
    delayed_task = recurrency_func(nested_session_dict[n_fact][activity_type][session], num_bootstrap, num_fits=num_fits)
    delayed_dict[params_to_calc] = delayed_task

    
### Begin calculatation
future_dict = client.compute(delayed_dict)

NameError: name 'session_dict' is not defined

In [ ]:
### Save results in xarray and to disc

result_dict = await future_dict.result()

res_dataset = xr.DataArray(data=None,  coords = [*zip(parameter_names, parameter_values)]+
                                                 [('variable',['var_cc', 'mean_var','mean_rho', 'taupost']),
                                                 ('trial_type',['HitAndMiss']),
                                                 ('realization',np.arange(num_fits))])
for params_res, res in result_dict.items():
    n_fact, session, activity_type = params_res
    var_cc_hit, mean_var_hit, mean_rho_hit, taupost_hit = res
    res_dataset.loc[dict([*zip(parameter_names, params_res)])] = np.array([var_cc_hit, mean_var_hit, mean_rho_hit, taupost_hit])

res_dataset.to_netcdf(f"../data/ML_LFA_nfac0,5_S1_TestHitAndMiss_taupost_results_{num_fits}fits.nc")

In [ ]:
## tau_post (pulse-based recurrence) analysis

In [ ]:
msm = MultiSessionModel(remove_targets=False, subsample_sessions=False,
                         remove_toosoon=True)

In [ ]:
def fit_func(t, A, tau):
    return A*np.exp(-t/tau)

In [ ]:
max_perms = 1000

hit_tau_hats = np.zeros((n_sessions, max_perms))
hit_tau_hats[:] = np.nan
miss_tau_hats = np.zeros((n_sessions, max_perms))
miss_tau_hats[:] = np.nan

for i_session in range(n_sessions):
    session = msm.sessions[i_session]
    
    targeted_neurons = np.any(session.is_target[:,:,0], axis=1)
    
    n_neurons = session.behaviour_trials.shape[0]
    n_frames = session.behaviour_trials.shape[2]
    
    noStim_trials = np.nonzero(session.photostim==0)[0]
    testStim_trials = np.nonzero(session.photostim==1)[0]
    easyStim_trials = np.nonzero(session.photostim==2)[0]
    
    hitTrials = np.nonzero(session.outcome=='hit')[0]
    missTrials = np.nonzero(session.outcome=='miss')[0]
    
    #testHitTrials = hitTrials
    #testMissTrials = missTrials
    
    testHitTrials = np.intersect1d(testStim_trials, hitTrials)
    testMissTrials = np.intersect1d(testStim_trials, missTrials)
    
    num_perms = int(comb(min([len(testHitTrials), len(testHitTrials)]), 10))
    
    if num_perms > max_perms:
        num_perms = max_perms
    
    for i_perm in range(num_perms):
        subHitTrials = np.random.choice(testHitTrials, 10, replace=False)
        subMissTrials = np.random.choice(testMissTrials, 10, replace=False)
    
        testHit_TargetNeuronAverages = np.zeros((n_neurons,n_frames))
        testHit_TargetNeuronAverages[:] = np.nan
        noHit_TargetNeuronAverages = np.zeros((n_neurons,n_frames))
        noHit_TargetNeuronAverages[:] = np.nan
        testMiss_TargetNeuronAverages = np.zeros((n_neurons,n_frames))
        testMiss_TargetNeuronAverages[:] = np.nan
        noMiss_TargetNeuronAverages = np.zeros((n_neurons,n_frames))
        noMiss_TargetNeuronAverages[:] = np.nan

        testHit_NonTargetNeuronAverages = np.zeros((n_neurons,n_frames))
        testHit_NonTargetNeuronAverages[:] = np.nan
        noHit_NonTargetNeuronAverages = np.zeros((n_neurons,n_frames))
        noHit_NonTargetNeuronAverages[:] = np.nan
        testMiss_NonTargetNeuronAverages = np.zeros((n_neurons,n_frames))
        testMiss_NonTargetNeuronAverages[:] = np.nan
        noMiss_NonTargetNeuronAverages = np.zeros((n_neurons,n_frames))
        noMiss_NonTargetNeuronAverages[:] = np.nan

        for i_neuron in np.arange(n_neurons)[targeted_neurons]:
            testHit_TargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subHitTrials[session.is_target[i_neuron,subHitTrials,0]]], axis=0)
            noHit_TargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subHitTrials[~session.is_target[i_neuron,subHitTrials,0]]], axis=0)
            testMiss_TargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subMissTrials[session.is_target[i_neuron,subMissTrials,0]]], axis=0)
            noMiss_TargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subMissTrials[~session.is_target[i_neuron,subMissTrials,0]]], axis=0)

        for i_neuron in np.arange(n_neurons)[~targeted_neurons]:
            testHit_NonTargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subHitTrials], axis=0)
            noHit_NonTargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subHitTrials], axis=0)
            testMiss_NonTargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subMissTrials], axis=0)
            noMiss_NonTargetNeuronAverages[i_neuron] = np.nanmean(session.behaviour_trials[i_neuron][subMissTrials], axis=0)

        testHit_delta = np.nanmean(testHit_TargetNeuronAverages, axis=0) - np.nanmean(np.nanmean(testHit_TargetNeuronAverages, axis=0)[:session.pre_frames])
        testMiss_delta = np.nanmean(testMiss_TargetNeuronAverages, axis=0)- np.nanmean(np.nanmean(testMiss_TargetNeuronAverages, axis=0)[:session.pre_frames])

        t = np.arange(0, (session.pre_frames+session.post_frames) - (session.pre_frames+250/30))
        testHit_popt, _ = curve_fit(fit_func, t, 
                                    testHit_delta[int(session.pre_frames+250/30):], 
                                    p0=(1, 100),
                                    maxfev = 10000)
        testMiss_popt, _ = curve_fit(fit_func, t, 
                                     testMiss_delta[int(session.pre_frames+250/30):], 
                                     p0=(1, 100),
                                     maxfev = 10000)
        
        hit_tau_hats[i_session,i_perm] = testHit_popt[1]
        miss_tau_hats[i_session,i_perm] = testMiss_popt[1]

In [ ]:
np.save('../data/PulseBasedRecurrencey_hit_tau_hats_1000MaxPerms.npy', hit_tau_hats)
np.save('../data/PulseBasedRecurrencey_miss_tau_hats_1000MaxPerms.npy', miss_tau_hats)